In [13]:
import copy
import glob
import json
import os
import random
import re
from random import shuffle

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

dataset_folder='./../dataset/saved/'

In [14]:
random.seed(0)

In [15]:
#root_dir = "/home/kirill/Documents/1.Projects/class-ranking/project-processing-results/processing/java/classes/processing/0.0.2/intellij-community/"
root_dir = "/home/kirill/Documents/1.Projects/class-ranking/ml-features/project-processing-results-2022.06.13.00.55-003/processing/java/classes/processing/0.0.3/intellij-community/"


def good(usage) -> bool:
    # {'TYPE', 'THIS_EXPRESSION', 'EXTENDS_LIST', 'IMPLEMENTS_LIST', 'METHOD_REF_EXPRESSION', 'EXPRESSION_LIST', 'NEW_EXPRESSION', 'CONDITIONAL_EXPRESSION', 'FIELD', 'ANNOTATION', 'THROWS_LIST', 'SUPER_EXPRESSION', 'JAVA_CODE_REFERENCE', 'REFERENCE_EXPRESSION', 'DOC_REFERENCE_HOLDER'}
    if usage["features"]["referenceType"] == "ANNOTATION":
        return False
    if usage["features"]["referenceType"] == "DOC_REFERENCE_HOLDER":
        return False
    return True

In [16]:
#ROOT_CLASS = "com.intellij.openapi.project.DumbAware"
ROOT_CLASS = "com.intellij.openapi.actionSystem.AnAction"
#print(familyUsages[ROOT_CLASS].most_common())

In [17]:
with open("paths.txt", 'r') as valid_list_file:
    valid_list = valid_list_file.readlines()
    valid_list = [s.rpartition('.')[0].rpartition('/')[2] for s in valid_list]

validClasses = set(valid_list)

In [18]:

eval_suggestions = {} # (enclosingClassName, offset) -> [suggestions]
eval_suggestions_classes = set() # {enclosingClassName}

cnt_all_sug_max = 4 * 10**5
cnt_all_sug = 0

for filename in glob.glob('./saved/eval_data/' + '**/*.json', recursive=True):
    enclosingClassName = filename.rpartition('/')[2].rpartition('.')[0].rpartition('.')[0].rpartition('_')[2]
    with open(filename, 'r') as f:
        data = json.load(f)
        sessions = data["sessions"]
        for session in sessions:
            suggestions = session["_lookups"][0]["suggestions"]
            suggestions_text = []
            for suggestion in suggestions:
                if ':' in suggestion["presentationText"]:
                    package = ''
                    name = suggestion["text"]
                else:
                    package = suggestion["presentationText"].rpartition(" ")[2].strip('()')
                    name = f'{package}.{suggestion["text"]}'

                # if '(' in suggestion["presentationText"]:
                #     print(suggestion)
                #     print(name)
                suggestions_text.append(name)
            eval_suggestions[(enclosingClassName, session['offset'])] = suggestions_text
            print(f'{(enclosingClassName, session["offset"])} | {len(suggestions_text)} {len(set(suggestions_text))}')
            eval_suggestions_classes.add(enclosingClassName)
            cnt_all_sug += len(suggestions_text)

        if cnt_all_sug > cnt_all_sug_max:
            break
    #print(f'{enclosingClassName}')

('EditorConfigActionUtil', 1429) | 661 655
('EditorConfigActionUtil', 1524) | 596 596
('EditorConfigActionUtil', 1568) | 561 561
('EditorConfigActionUtil', 1588) | 621 621
('EditorConfigActionUtil', 1658) | 622 622
('EditorConfigActionUtil', 1832) | 654 654
('EditorConfigActionUtil', 2035) | 213 213
('EditorConfigActionUtil', 2163) | 621 621
('EditorConfigActionUtil', 2195) | 595 595
('EditorConfigActionUtil', 2315) | 597 597
('EditorConfigActionUtil', 2961) | 660 659
('EditorConfigActionUtil', 3096) | 599 599
('EditorConfigActionUtil', 3313) | 213 213
('EditorConfigActionUtil', 3317) | 585 585
('EditorConfigActionUtil', 3363) | 213 211
('EditorConfigActionUtil', 3413) | 598 598
('EditorConfigActionUtil', 3600) | 525 525
('EditorConfigActionUtil', 3739) | 563 563
('EditorConfigActionUtil', 3797) | 606 606
('EditorConfigActionUtil', 4004) | 599 599
('EditorConfigActionUtil', 4068) | 601 601
('EditorConfigActionUtil', 4439) | 573 573
('EditorConfigActionUtil', 4816) | 213 213
('EditorCon

In [19]:
def listToStr(df, col):
    tmp = df[col].isnull(), col
    df.loc[tmp] = df.loc[tmp].apply(lambda x: [])
    df[col] = df[col].apply(lambda x: " ".join(x) )

In [20]:
print(eval_suggestions_classes)

{'DexFileViewer', 'AbstractCommonUpdateAction', 'FocusOnStartAction', 'CloseEditorsActionBase', 'CreateResourceDirectoryAction', 'CustomizableActionsPanel', 'CheckSanityAction', 'GoToModuleAction', 'EditorConfigActionUtil', 'GradleExecuteTaskAction', 'ChooseDeploymentAction', 'AddLibraryToModuleDependenciesAction', 'GitMerge', 'ApplyPatchViewer', 'GitDiffFromHistoryHandler', 'ExternalSystemNodeAction'}


In [21]:

ccc = 0
cnt = 0
cntu = 0
list_of_dfs_valid = []
group_id = 0
for filename in glob.glob(root_dir + '**/*.json', recursive=True):
    ccc += 1
    enclosingClassName = filename.rpartition('/')[0].rpartition('/')[2]
    # if ccc < 10:
    #     print(f'!! = {enclosingClassName}')
    if enclosingClassName.rpartition('.')[2] not in validClasses:
        continue

    # print(f'?? = {enclosingClassName}')

    if enclosingClassName.rpartition('.')[2] not in eval_suggestions_classes:
            continue

    # print(f'enclosingClassName = {enclosingClassName}')

    with open(os.path.join(os.getcwd(), filename), 'r') as f:
        data = json.load(f)
        data_usages = []
        # if enclosingClassName in validClasses:
        #     with open("paths.txt", 'a') as log:
        #         log.write(f'\"{data["usages"][0]["filePath"]}\",\n')
        for usage in data["usages"]:
            if not good(usage):
                continue
            usage['enclosingClassExtendsList'] = data['keyInfo']['additionalInfo']['enclosingClassExtendsList']
            usage['enclosingClassImplementsList'] = data['keyInfo']['additionalInfo']['enclosingClassImplementsList']
            usage['correct'] = 1
            usage['firstChar'] = usage['name'].rpartition('.')[2][0]
            usage['group'] = group_id
            group_id += 1

            if (enclosingClassName.rpartition('.')[2], usage['textOffset']) not in eval_suggestions:
                # print(f'{(enclosingClassName, usage["textOffset"])} - skip')
                continue

            # print((enclosingClassName, usage['textOffset']))

            data_usages.append(usage)
            for sug in eval_suggestions[(enclosingClassName.rpartition('.')[2], usage['textOffset'])]:
                #print(f" {(enclosingClassName.rpartition('.')[2], usage['textOffset'])} {usage['name'].rpartition('.')[2]} vs {sug}")
                if sug == usage['name'].rpartition('.')[2]:
                    #print(f" -------- {(enclosingClassName.rpartition('.')[2], usage['textOffset'])} {usage['name'].rpartition('.')[2]}")
                    continue
                usage_negative = copy.copy(usage)
                usage_negative['name'] = sug
                usage_negative['correct'] = 0 # а правильный
                data_usages.append(usage_negative)

        if len(data_usages) == 0:
            continue
        df = pd.json_normalize(data_usages)
        df['features.enclosingClassName'] = enclosingClassName

        # print(enclosingClassName)
        # if enclosingClassName == 'com.intellij.openapi.project.DumbAware' :
        #     print(df.dtypes)
        df = df.drop(columns=['filePath', 'textOffset', 'features.lineInFile', 'features.lineInScope'], errors='ignore')
        if enclosingClassName.rpartition('.')[2] in validClasses:
            list_of_dfs_valid.append(df)
        cntu += len(data_usages)
        cnt += 1

df_valid_raw = pd.concat(list_of_dfs_valid, axis=0, ignore_index=True)

listToStr(df_valid_raw, 'features.variablesTypesInScope')
listToStr(df_valid_raw, 'features.variablesNamesInScope')
listToStr(df_valid_raw, 'features.variablesTypesInEnclosingClassRemaining')
listToStr(df_valid_raw, 'enclosingClassExtendsList')
listToStr(df_valid_raw, 'enclosingClassImplementsList')
df_valid_raw.fillna('', inplace=True)

print(f'{cnt} files loaded, {cntu} usages')

print(f'valid shape {df_valid_raw.shape}')
df_valid_raw.dtypes

# will we know referenceType irl ?

14 files loaded, 134354 usages
valid shape (134354, 16)


name                                                object
enclosingClassExtendsList                           object
enclosingClassImplementsList                        object
correct                                              int64
firstChar                                           object
group                                                int64
features.referenceType                              object
features.scopeKind                                  object
features.enclosingScopeName                         object
features.variablesTypesInEnclosingClassRemaining    object
features.variablesTypesInScope                      object
features.variablesNamesInScope                      object
features.insideStatement_0                          object
features.insideStatement_1                          object
features.insideStatement_2                          object
features.enclosingClassName                         object
dtype: object

In [22]:
df_valid_raw.to_csv(dataset_folder+'valid_eval_raw_2.csv')

In [23]:
df_valid_raw2 = pd.read_csv(dataset_folder+'valid_eval_raw_2.csv')


/tmp/ipykernel_103216/2605763236.py:1: DtypeWarning: Columns (2,3,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_valid_raw2 = pd.read_csv(dataset_folder+'valid_eval_raw_2.csv')


In [24]:
df_valid_raw2.columns

Index(['Unnamed: 0', 'name', 'enclosingClassExtendsList',
       'enclosingClassImplementsList', 'correct', 'firstChar', 'group',
       'features.referenceType', 'features.scopeKind',
       'features.enclosingScopeName',
       'features.variablesTypesInEnclosingClassRemaining',
       'features.variablesTypesInScope', 'features.variablesNamesInScope',
       'features.insideStatement_0', 'features.insideStatement_1',
       'features.insideStatement_2', 'features.enclosingClassName'],
      dtype='object')